In [1]:
import pandas as pd
import json
import requests
from ast import literal_eval

In [2]:
pd.set_option("max_colwidth", 1000)
data = pd.read_csv("course_list.csv", index_col=0)
print(data.shape)
data.head()

(9738, 16)


,academic_disciplines,campus,category,course_class_code,course_code,credits,eligible_year,instructors,level,main_language,school,sessions,syllabus_urls,term,title,year
0,"['Literature', 'Literature', 'Introduction']",waseda,Humanities,1,LITV101L,2.0,1st year and above,"['YAGI, Naoko']","Beginner, initial or introductory",English,School of Political Science and Economics,"[{'period': '4', 'classrooms': ['3-601'], 'day': 'Tues'}]",['https://www.wsl.waseda.jp/syllabus/JAA104.php?pLng=en&pKey=11000012G101201911000012G111'],spring semester,World Literature [E] 01,2019
1,"['Art Studies', 'Theatre', 'Others']",waseda,Humanities,1,ARTT181L,2.0,1st year and above,"['YAGI, Naoko']","Beginner, initial or introductory",English,School of Political Science and Economics,"[{'period': '4', 'classrooms': ['3-709'], 'day': 'Thur'}]",['https://www.wsl.waseda.jp/syllabus/JAA104.php?pLng=en&pKey=11000012G301201911000012G311'],fall semester,Drama and Society [E] 01,2019
2,"['Art Studies', 'Theatre', 'Others']",waseda,Seminarｓ: Intermediate Courses,1,ARTT281L,2.0,1st year and above,"['JEWEL, Mark']","Intermediate, developmental and applicative",English,School of Political Science and Economics,"[{'period': '3', 'classrooms': ['3-713'], 'day': 'Fri'}]",['https://www.wsl.waseda.jp/syllabus/JAA104.php?pLng=en&pKey=1100001S160120191100001S1611'],fall semester,Intermediate Seminar (Postwar Japanese Fiction in English Translation) [E] 01,2019
3,"['Art Studies', 'Theatre', 'Others']",waseda,Seminarｓ: Intermediate Courses,1,ARTT281L,2.0,1st year and above,"['JEWEL, Mark']","Intermediate, developmental and applicative",English,School of Political Science and Economics,"[{'period': '3', 'classrooms': ['3-812'], 'day': 'Mon'}]",['https://www.wsl.waseda.jp/syllabus/JAA104.php?pLng=en&pKey=1100001S170120191100001S1711'],fall semester,Intermediate Seminar (Masters of Haiku) [E] 01,2019
4,"['Art Studies', 'Theatre', 'Others']",waseda,Seminarｓ: Intermediate Courses,1,ARTT281L,2.0,2nd year and above,"['JEWEL, Mark']","Intermediate, developmental and applicative",English,School of Political Science and Economics,"[{'period': '3', 'classrooms': ['3-609'], 'day': 'Fri'}]",['https://www.wsl.waseda.jp/syllabus/JAA104.php?pLng=en&pKey=1100001S180120191100001S1811'],spring semester,Intermediate Seminar (Prewar Japanese Fiction in English Translation) [E] 01,2019


In [3]:
data["academic_disciplines"] = data["academic_disciplines"].apply(literal_eval)
data["instructors"] = data["instructors"].apply(literal_eval)
data["sessions"] = data["sessions"].apply(literal_eval)
data["schools"] = data["school"].apply(lambda x : [x])
data["syllabus_url"] = data["syllabus_urls"].apply(literal_eval).apply(lambda x : x[0])
data = data.drop(columns=["syllabus_urls", "school"])

data["year"] = data["year"].apply(str)
data = data.fillna("\u00a0")

In [4]:
assert len(data.iloc[[0]]["sessions"]) == 1

In [5]:
seen_courses = {}
for index, row in data.iterrows() :
    serial = str(row["title"]) + str(row["sessions"]) + str(row["term"]) + str(row["instructors"])
    if serial in seen_courses.keys() :
        if data.loc[[index]]["schools"].tolist()[0][0] not in data.loc[[seen_courses[serial]]]["schools"].tolist()[0] :
            data.loc[[seen_courses[serial]]]["schools"] = pd.Series(data.loc[[seen_courses[serial]]]["schools"].tolist()[0].append(data.loc[[index]]["schools"].tolist()[0][0]))
        data = data.drop([index])
    else :
        seen_courses[serial] = index
    

In [6]:
assert data.loc[data["title"] == "Signal Processing"].shape[0] == 1
assert data.loc[data["title"] == "Calculus A"].shape[0] == 1

In [7]:
data.dtypes

academic_disciplines     object
campus                   object
category                 object
course_class_code        object
course_code              object
credits                 float64
eligible_year            object
instructors              object
level                    object
main_language            object
sessions                 object
term                     object
title                    object
year                     object
schools                  object
syllabus_url             object
dtype: object

In [8]:
print(data.isnull().sum(axis=0))

academic_disciplines    0
campus                  0
category                0
course_class_code       0
course_code             0
credits                 0
eligible_year           0
instructors             0
level                   0
main_language           0
sessions                0
term                    0
title                   0
year                    0
schools                 0
syllabus_url            0
dtype: int64


In [9]:
print("In academic_disciplines,  max list length: ", data["academic_disciplines"].str.len().max())
print("max str length: ", data["academic_disciplines"].apply(lambda x: max([len(item) for item in x])).max(), "\n")
print("In campus, max str length: ", data["campus"].str.len().max(), "\n")
print("In category, max str length: ", data["category"].str.len().max(), "\n")
print("In course_code, max str length: ", data["course_code"].str.len().max(), "\n")
print("In instructors, max list length: ", data["instructors"].str.len().max())
print("max str length: ", data["instructors"].apply(lambda x: max([len(item) for item in x])).max(), "\n")
print("In title, max str length: ", data["title"].str.len().max())
print("In schools,  max list length: ", data["schools"].str.len().max())
print("max str length: ", data["schools"].apply(lambda x: max([len(item) for item in x])).max(), "\n")

In academic_disciplines,  max list length:  3
max str length:  80 

In campus, max str length:  27 

In category, max str length:  83 

In course_code, max str length:  8 

In instructors, max list length:  43
max str length:  43 

In title, max str length:  127
In schools,  max list length:  3
max str length:  66 



In [17]:
def write_to_local(data) :
    data.to_csv("course_list_cleaned.csv")

In [11]:
print(data.shape)

write_to_local(data)

(7727, 16)
